# Proyek Akhir Robotika


## Anggota:
- Anel
- Adnan
- Rio
- Tegar
- Alif

## Planning

### Identifikasi Masalah


1. Masalah Utama (Tracking Bola):
    - Bagaimana mengembangkan robot yang mampu mendeteksi dan melacak bola secara akurat dalam berbagai kondisi pencahayaan dan lingkungan?
    - Tantangan: Deteksi bola harus real-time dan konsisten, sementara robot harus bisa menggerakkan dirinya ke arah bola dengan responsif menggunakan motor DC.
2. Optimasi Tracking:Masalah: 
    - Bagaimana memastikan robot bisa melacak bola secara sempurna dan menghindari kesalahan deteksi atau pelacakan, seperti kehilangan target atau kesalahan gerakan robot saat mengikuti bola.
    -Tantangan: Perlu penyesuaian algoritma deteksi dan gerakan yang dinamis untuk berbagai kecepatan dan arah pergerakan bola.
3. Fitur Tambahan (Menendang dan Mengenal Gawang)(Optional, jika 2 poin diatas sudah diatasi):
    - Masalah: Setelah pelacakan bola sempurna, bagaimana menambahkan fitur agar robot bisa mengenali dan menendang bola ke arah target, serta mengenali gawang sebagai tujuan akhir.
    - Tantangan: Implementasi model tambahan atau pengenalan objek lebih kompleks, seperti deteksi gawang, serta kontrol motorik yang lebih halus untuk menendang bola dengan kekuatan dan akurasi yang tepat.

### Modeling

1. Modeling (Deteksi Warna Awal)
Kami akan menggunakan kamera dan HSV color space untuk mendeteksi warna bola secara real-time. Motor DC akan menggerakkan robot sesuai dengan posisi bola yang terdeteksi, dengan algoritma yang menggunakan informasi posisi X dan Y bola.

2. Modeling Lanjutan (YOLO)
Ketika deteksi warna sudah berjalan dengan baik, kami akan beralih ke metode YOLO untuk mendeteksi bola secara lebih presisi dan dalam kondisi pencahayaan yang lebih kompleks. YOLO akan dilatih menggunakan dataset dengan label bounding box yang bisa kami peroleh dari Kaggle atau melalui pelabelan manual. Setelah model YOLO dilatih, kami akan mengimplementasikan untuk mendeteksi bola dalam video secara real-time. Posisi bounding box bola yang terdeteksi akan digunakan untuk menggerakkan robot secara akurat mengikuti bola.


### Data Gathering

1. Data Gathering
Untuk tahap awal, kami dapat merekam video bola dengan warna yang mencolok dan kontras tinggi di berbagai kondisi pencahayaan. Setelah video diambil, kami akan mengekstrak beberapa frame untuk melatih dan menguji model. Frame yang diambil akan dilabeli warna bola untuk membantu sistem dalam mengenali targetnya.

2. Dataset Kaggle
Selain itu, kami juga dapat memanfaatkan dataset yang sudah ada di Kaggle untuk mempercepat proses. Contoh dataset yang relevan bisa ditemukan di Kaggle, seperti dataset bola sepak yang sudah dilabeli. Dataset dari Kaggle akan sangat membantu dalam fase pelatihan, baik untuk deteksi berbasis warna maupun YOLO.

### Preprocessing Data

1. Preprocessing Data
Kami akan mengonversi citra dari format RGB ke HSV agar lebih mudah dalam mendeteksi warna spesifik dari bola. Selanjutnya, kami terapkan thresholding pada warna bola untuk membuat area bola lebih jelas dan terisolasi dari latar belakang.
2. Preprocessing Lanjutan
Jika menggunakan dataset gambar, tentunya preprocessing akan terdiri dari resizing, pengahpusan noise. Tak kalah penting akan dilakukan labeling dengan bounding box untuk memberikan info posisi bola sebelum di latih

### Ekstraksi Fitur

1. Ekstraksi Fitur
Setelah deteksi warna dilakukan, kami akan mengekstrak posisi bola dengan menghitung centroid dari area bola yang terdeteksi. Posisi X dan Y dari centroid ini akan digunakan untuk mengontrol arah gerakan robot secara otomatis, menggerakkannya ke arah bola.

## Excecution

### Progress Pertama (Kode Mengenal Bola dengan Contour Warna)

In [5]:
import cv2
import imutils
import numpy as np

# Mendefinisikan nilai rentang HSV untuk warna bola (biru dan hijau)
ball_hsv_values = {
    "Blue": {  # Rentang HSV untuk bola biru
        "lower": (89, 100, 100),  
        "upper": (109, 255, 255)
    },
    "Green": {  # Rentang HSV untuk bola hijau
        "lower": (61, 100, 100),
        "upper": (81, 255, 255) 
    }
}




In [6]:
# Kelas BallTracker 
class BallTracker:
    MIN_BALL_RADIUS = 10  
    MAX_BALL_RADIUS = 60
    FRAME_WIDTH = 500 

    # Konstruktor
    def __init__(self, color: str) -> None:
        assert color in ball_hsv_values 
        self.color = color
        self.lower_bound = ball_hsv_values[color]['lower']  
        self.upper_bound = ball_hsv_values[color]['upper'] 
        self.camera = cv2.VideoCapture(0)  

    # Fungsi untuk mengambil frame video dan mengubahnya ke format HSV
    def get_video_frame(self):
        ret, frame_in_bgr = self.camera.read()  
        if not ret:
            raise Exception('Unable to read from the camera')  
        frame_in_bgr = imutils.resize(frame_in_bgr, width=self.FRAME_WIDTH) 
        frame_in_hsv = cv2.cvtColor(frame_in_bgr, cv2.COLOR_BGR2HSV)  
        return frame_in_bgr, frame_in_hsv  

    # Fungsi untuk membuat mask dari frame berdasarkan rentang warna HSV
    def get_mask(self, frame_in_hsv):
        mask = cv2.inRange(frame_in_hsv, self.lower_bound, self.upper_bound)  
        mask = cv2.erode(mask, None, iterations=2) 
        mask = cv2.dilate(mask, None, iterations=2) 
        return mask

    # Fungsi untuk menemukan kontur terbesar dari mask (yang diasumsikan sebagai bola)
    def get_contour(self, mask):
        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0]  
        if len(contours) > 0:
            return max(contours, key=cv2.contourArea)  # Mengembalikan kontur terbesar (asumsi sebagai bola)
        return None  

    # Fungsi utama
    def track_ball(self, display_on_screen=True):
        while True:
            frame_in_bgr, frame_in_hsv = self.get_video_frame()
            mask = self.get_mask(frame_in_hsv) 
            contour = self.get_contour(mask)

            # Menggambar sistem koordinat Kartesius di tengah frame
            height, width, _ = frame_in_bgr.shape
            cv2.line(frame_in_bgr, (width // 2, 0), (width // 2, height), (255, 0, 0), 1)
            cv2.line(frame_in_bgr, (0, height // 2), (width, height // 2), (255, 0, 0), 1)
            
            if contour is not None:
            # Mencari lingkaran yang melingkupi kontur dan mengambil posisinya
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                if radius > self.MIN_BALL_RADIUS:
                    x, y, radius = int(x), int(y), int(radius)  # Konversi ke integer
                    cv2.circle(frame_in_bgr, (x, y), radius, (0, 0, 255), 2)  # Gambar lingkaran di sekitar bola

                    # Periksa apakah bola cukup besar dan berada di tengah
                    if radius > 100 and (width // 2 - 50 < x < width // 2 + 50):
                        command = "Tendang"
                    else:
                        command = ""  # Kosongkan jika tidak ada perintah tendang

                    # Tentukan arah gerakan robot berdasarkan posisi bola
                    if x < width // 2 - 50:  # Bola berada di kiri
                        movement = "Turn Left"
                    elif x > width // 2 + 50:  # Bola berada di kanan
                        movement = "Turn Right"
                    else:  # Bola berada di tengah
                        movement = "Move Forward"

                    # Tampilkan instruksi gerakan pada frame
                    cv2.putText(frame_in_bgr, movement, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    # Tampilkan perintah tendang jika ada
                    if command:
                        cv2.putText(frame_in_bgr, command, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            if display_on_screen:
                cv2.imshow("Ball Tracking", frame_in_bgr)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        self.camera.release()
        cv2.destroyAllWindows()


In [7]:
tracker = BallTracker("Blue") 
tracker.track_ball()

In [9]:
tracker = BallTracker("Green") 
tracker.track_ball()

Kode di atas bertujuan untuk melacak bola menggunakan deteksi warna berbasis HSV dan menggerakkan robot berdasarkan posisi bola. Jika kontur bola terdeteksi, sistem menghitung lingkaran yang mengelilingi bola untuk menentukan posisi X dan Y serta radius bola. Berdasarkan posisi bola di frame, robot akan mengambil tindakan seperti "Turn Left", "Turn Right", atau "Move Forward". Jika bola cukup besar dan berada di tengah frame, robot akan mengeluarkan perintah "Tendang". Perintah ini serta instruksi gerakan ditampilkan pada video yang diproses, memungkinkan pengendalian robot secara real-time.